# Что будет
- сводные таблицы
- фильтры и вычисления с помощью метода loc
- фильтрация пустых значений через isnull
- время в pandas
- строковые методы
- немного про учет форм слов

# Сводные таблицы
Прям как в экселе

In [1]:
import pandas as pd

In [4]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### Упражнение
Какие оценки поставил пользователь №1 и в каком количестве?

Что делаем:
1. Фильтруем датафрейм ratings для userId = 1
2. Считаем для этого пользователя сколько он выставил единиц, двоек итд

In [4]:

rating = ratings[ ratings['userId'] == 1].groupby('rating').count()
rating

,userId,movieId,timestamp
rating,,,
1.0,2,2,2
2.0,7,7,7
2.5,3,3,3
3.0,4,4,4
3.5,1,1,1
4.0,3,3,3


Object `pd.count()` not found.


In [10]:
?ratings.pivot_table

Как это сделать для всех пользователей сразу:

In [8]:
ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0).head()


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,2,0,7,3,4,1,3,0,0
2,0,2,0,4,0,36,0,23,0,11
3,0,0,0,1,3,18,9,11,4,5
4,0,5,0,5,0,23,0,52,0,119
5,0,0,1,0,3,3,27,42,19,5


In [21]:
# можно итоги добавить
ratings[ratings['rating'] == 5].pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0, 
                    margins = True).head()

rating,5.0,All
userId,,
2,11,11
3,5,5
4,119,119
5,5,5
6,4,4


### Упражнение
Какой пользователь выставил больше всех пятерок?

In [32]:
ratings[ratings['rating'] == 5].pivot_table(index='userId', values='timestamp', columns='rating',aggfunc='count', fill_value = 0, margins=True).sort_values(by=5, ascending = False)

rating,5.0,All
userId,,
All,15095,15095
564,408,408
232,243,243
242,219,219
547,214,214
...,...,...
552,1,1
50,1,1
504,1,1


In [26]:
ratings[ratings['rating'] == 5].sort_values(by = 'rating', ascending = True)

,userId,movieId,rating,timestamp
21,2,17,5.0,835355681
68136,473,68157,5.0,1425645591
68137,473,72998,5.0,1425643504
68138,473,76093,5.0,1425643337
68140,473,79132,5.0,1425643321
...,...,...,...,...
33400,240,3751,5.0,1098941805
33410,240,3967,5.0,1098941840
33478,240,7090,5.0,1098940849
33239,239,3548,5.0,991857857


### Фильтры и вычисления с помощью loc и iloc

In [28]:
log = pd.read_csv('visit_log.csv', sep=';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


### Но сначала упражнение
Сколько различных страниц есть в столбце url? Постройте топ-5 страниц по посещаемости.

Бонусный вариант: выведите топ-5 страниц без указания хоста https://host.ru

In [43]:
log.groupby('url').count().sort_values(by='user_id', ascending = False).head()

,timestamp,visit_id,region,user_id,traffic_source
url,,,,,
https://host.ru/3c19b4ef7371864fa3,541,541,541,541,541
https://host.ru/108ce4b365afb7b88e,534,534,534,534,534
https://host.ru/23c64743ad842b9311,532,532,532,532,532
https://host.ru/1afc5618562365020a,529,529,529,529,529
https://host.ru/2fcb223266880ccf57,526,526,526,526,526


In [44]:
urlpars

NameError: name 'urlpars' is not defined

In [29]:
# метод loc позволяет выбрать строки и столбцы в соответствии с условиями
# двоеточие означает выбор всех значений

log.loc[:, ['user_id', 'region']].head()

,user_id,region
0,b1613cc09f,Russia
1,4c3ec14bee,Russia
2,a8c40697fb,Russia
3,521ac1d6a0,Russia
4,d7323c571c,Russia


In [31]:
# iloc работает аналогично, но с указанием номера строки / столбца

log.iloc[:, [1, 2]].head()

,visit_id,url
0,e3b0c44298,https://host.ru/3c19b4ef7371864fa3
1,6e340b9cff,https://host.ru/c8d9213a31839f9a3a
2,96a296d224,https://host.ru/b8b58337d272ee7b15
3,709e80c884,https://host.ru/b8b58337d272ee7b15
4,df3f619804,https://host.ru/b8b58337d272ee7b15


In [35]:
# пример фильтра на страну
# если столбцы не надо фильтровать, то второй параметр можно не указывать

log.loc[log.region == 'Russia'].head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [51]:
# пример вычисления нового столбца с НДС для страны

log.loc[log.region == 'Russia', 'VAT'] = 1.2
log.head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2


In [52]:
# вариант с вычисляемым фильтром

log.loc[lambda row: row.region == 'Russia'].head(10)

,timestamp,visit_id,url,region,user_id,traffic_source,VAT
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,1.2
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,1.2
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,1.2
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,1.2
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,1.2
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,1.2
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,1.2
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,1.2
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,1.2
12,1549980789,15ec7bf0b5,https://host.ru/8e88d4703848cc0ec4,Russia,6d1d81f7f8,paid,1.2


### Упражнение
Какие варианты источников трафика есть в столбце traffic_source?

Создайте столбец traffic_type, в котором для источников 'yandex' и 'google' будет стоять значение 'organic'. А для остальных вариантов - NaN.

In [49]:
log['traffic_source'].unique()

array(['yandex', 'direct', 'paid', 'email', 'google'], dtype=object)

In [50]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [60]:
log.loc[log.traffic_source.isin(['yandex','google']), 'traffic_type' ] = 'organic'
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


### Методы isnull, isna
Определение пустых или None значений. По сути одинаковые методы

In [2]:
import numpy as np

In [6]:
df = pd.DataFrame({'value': [123, None, np.nan, np.NaN, np.NAN, 456]})
df

,value
0,123.0
1,NaN
2,NaN
3,NaN
4,NaN
5,456.0


In [40]:
# фильтр на пустые значения в столбце value

df.loc[pd.isnull(df.value), :]

,value
1,NaN
2,NaN
3,NaN
4,NaN


In [42]:
df.loc[~pd.isnull(df.value), :]

,value
0,123.0
5,456.0


### Упражнение
Для пустых значений в столбце traffic_type выставьте значение 'other'

In [61]:
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [64]:
log.loc[pd.isnull(log.traffic_type),'traffic_type'] = 'other'

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,other
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,other
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,other
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,other


In [70]:
log.region.head()

0    Russia
1    Russia
2    Russia
3    Russia
4    Russia
Name: region, dtype: object

### Дата и время в pandas

In [73]:
log['date'] = pd.to_datetime(log['timestamp'], unit='s')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,traffic_type,date
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic,2019-02-12 14:11:32
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,other,2019-02-12 14:11:44
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic,2019-02-12 14:11:55
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic,2019-02-12 14:12:05
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic,2019-02-12 14:12:16


In [72]:
# столбец datetime64[ns] теперь имеет тип даты
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18938 entries, 0 to 18937
Data columns (total 8 columns):
timestamp         18938 non-null int64
visit_id          18938 non-null object
url               18938 non-null object
region            18938 non-null object
user_id           18938 non-null object
traffic_source    18938 non-null object
traffic_type      18938 non-null object
date              18938 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.2+ MB


In [ ]:
# получим час визита

log['hour'] = log.date.dt.hour
log.head()

### Упражнение
Дан датафрейм df с временем визита и соответствующей покупки. Сделайте следующее:
1. Создайте столбцы visit_dt и purchase_dt типа datetime из столбцов visit_datetime и purchase_datetime
2. Посчитайте разницу между временем визита и покупки (простым вычитанием)
3. Переведите полученный столбец с длительностью покупки в минуты:
```python
(df.purchase_dt - df.visit_dt).astype('timedelta64[m]')
```
4. Посчитайте среднее время покупки в минутах

In [74]:
df = pd.DataFrame({'visit_datetime': ['2019-11-04T00:05:13', '2019-11-04T00:06:19', '2019-11-04T01:35:14'], 
                   'purchase_datetime': ['2019-11-04T00:15:53', '2019-11-04T00:07:59', '2019-11-04T03:15:30']})
df

,visit_datetime,purchase_datetime
0,2019-11-04T00:05:13,2019-11-04T00:15:53
1,2019-11-04T00:06:19,2019-11-04T00:07:59
2,2019-11-04T01:35:14,2019-11-04T03:15:30


In [81]:
df.loc[:, 'visit_dt'] = df.loc[:, 'visit_datetime']
df.loc[:, 'purchase_dt'] = df.loc[:, 'purchase_datetime']
df['delay'] = (df.purchase_dt - df.visit_dt).astype('timedelta64[m]')


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [79]:
df.head()

,visit_datetime,purchase_datetime,visit_dt,purchase_dt
0,2019-11-04T00:05:13,2019-11-04T00:15:53,2019-11-04T00:05:13,2019-11-04T00:15:53
1,2019-11-04T00:06:19,2019-11-04T00:07:59,2019-11-04T00:06:19,2019-11-04T00:07:59
2,2019-11-04T01:35:14,2019-11-04T03:15:30,2019-11-04T01:35:14,2019-11-04T03:15:30


### Методы работы со строками

In [88]:
stats = pd.read_csv('keywords.csv')
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [90]:
import pandas as pd
import numpy as np
stats = pd.read_csv('keywords.csv')
stats[ stats['keyword'].str.contains('москва')]

,keyword,shows
127,авито москва,979292
1259,москва,153050
1396,гисметео москва,140279
1482,госуслуги москва,133061
1682,погода москва,151701
...,...,...
98388,пилюли ру интернет аптека москва,3690
98679,геи москва,4205
98943,вакансии на авито москва от прямых работодателей,3738
99292,афиша кино москва на сегодня,3704


Проверка наличия подстроки в строке в питоне:

In [91]:
'охотник' in 'каждый охотник желает знать...'

True

Аналог в pandas:

In [92]:
stats[stats.keyword.str.contains('охотник')].head()

,keyword,shows
3072,сумеречные охотники 2 сезон,71965
3474,сумеречные охотники,66083
3654,белоснежка и охотник 2 фильм 2016,63154
4178,последний охотник на ведьм,57560
6127,последний охотник на ведьм фильм 2015,42213


[Документация](https://www.geeksforgeeks.org/python-pandas-series-str-contains/)

Syntax: Series.str.contains(pat, case=True, flags=0, na=nan, regex=True)

Parameter :
- pat : Character sequence or regular expression.
- case : If True, case sensitive.
- flags : Flags to pass through to the re module, e.g. re.IGNORECASE.
- na : Fill value for missing values.
- regex : If True, assumes the pat is a regular expression.

In [93]:
# поиск одного из нескольких слов

stats[stats.keyword.str.contains('охотник|фильм|2016')].head()

,keyword,shows
20,фильмы 2016,4486635
51,фильмы,2156641
54,фильмы онлайн,2305540
68,смотреть фильмы онлайн,1928484
86,порно фильмы,1458031


### Упражнение
Отфильтруйте датафрейм stats по поисковым запросам, которые содержат строку "погода в" и упоминают один из городов: Москва, Новосибирск, Краснодар.

In [94]:
stats.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [99]:
stats[ stats['keyword'].str.contains('погода в' and 'москва|новосибирск|краснодар')]

,keyword,shows
127,авито москва,979292
536,авито краснодарский край,309429
633,дром новосибирск,270598
651,погода в краснодаре,261285
654,авито краснодар,264400
...,...,...
98779,галерея новосибирск,3732
98943,вакансии на авито москва от прямых работодателей,3738
99059,e2e4 новосибирск,3665
99292,афиша кино москва на сегодня,3704


### replace

In [100]:
'отпуск начнется завтра'.replace('завтра', 'через месяц')

'отпуск начнется через месяц'

Аналог в pandas на запросах про сериалы:

In [101]:
serial = stats[stats.keyword.str.contains('сериалы')]
serial.head()

,keyword,shows
246,сериалы,587779
304,сериалы тут,503967
555,турецкие сериалы на русском языке,296403
881,русские сериалы,234262
890,сериалы онлайн,204812


In [102]:
serial.keyword.str.replace('сериалы', 'книги').head()

246                              книги
304                          книги тут
555    турецкие книги на русском языке
881                      русские книги
890                       книги онлайн
Name: keyword, dtype: object

### Как учитывать разное написание слов
Самое простое - методы upper и lower

In [ ]:
serial.keyword.str.upper().head()

In [ ]:
serial.keyword.str.lower().head()

### Что делать если нужно учесть формы написания слов?

In [ ]:
stats[stats.keyword.str.contains('рубл')].head()

Почему нельзя просто оставить str.contains('рубл'):

In [ ]:
non_financial_search = 'рубленая котлетка'

### Библиотека [pymystem](https://pypi.org/project/pymystem3/)

In [6]:
from pymystem3 import Mystem

In [7]:
search = 'курс гривны к рублю рубли рублях'

In [8]:
m = Mystem()
lemmas = m.lemmatize(search)
lemmas

Installing mystem to /Users/anthony/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


['курс',
 ' ',
 'гривна',
 ' ',
 'к',
 ' ',
 'рубль',
 ' ',
 'рубль',
 ' ',
 'рубль',
 '\n']

In [14]:
' '.join(lemmas)

'курс   гривна   к   рубль   рубль   рубль \n'

In [16]:
' '.join(Mystem().lemmatize(search))

'курс   гривна   к   рубль   рубль   рубль \n'

### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [ ]:
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})